# Important stuff

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data Download

In [ ]:
! pip install -q kaggle

In [ ]:
# using kaggle.json
from google.colab import files
files.upload()

In [ ]:
# Hiding kaggle.json file and downloading 
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c competitive-data-science-predict-future-sales

mkdir: cannot create directory ‘/root/.kaggle’: File exists
  0% 0.00/2.91k [00:00<?, ?B/s]
100% 2.91k/2.91k [00:00<00:00, 2.62MB/s]
 83% 11.0M/13.3M [00:00<00:00, 115MB/s]
100% 13.3M/13.3M [00:00<00:00, 120MB/s]
  0% 0.00/468k [00:00<?, ?B/s]
100% 468k/468k [00:00<00:00, 150MB/s]
  0% 0.00/368k [00:00<?, ?B/s]
100% 368k/368k [00:00<00:00, 123MB/s]
  0% 0.00/3.49k [00:00<?, ?B/s]
100% 3.49k/3.49k [00:00<00:00, 3.63MB/s]
  0% 0.00/1.02M [00:00<?, ?B/s]
100% 1.02M/1.02M [00:00<00:00, 134MB/s]


In [ ]:
import os

my_dir = '/content/'
filelist = [f for f in os.listdir(my_dir) if '.' in f]
filelist = filelist[2:]
print(filelist)

['sales_train.csv.zip', 'items.csv.zip', 'sample_submission.csv.zip', 'item_categories.csv', 'shops.csv', 'test.csv.zip']


In [ ]:
! mkdir '/content/drive/MyDrive/HSE Final Project'
! mkdir '/content/drive/MyDrive/HSE Final Project/Data'

In [ ]:
import shutil

new_dir = '/content/drive/MyDrive/HSE Final Project/Data/'

for file_name in filelist:
    shutil.move(my_dir + file_name, new_dir + file_name)

In [ ]:
! unzip '/content/drive/MyDrive/HSE Final Project/Data/items.csv.zip'
! unzip '/content/drive/MyDrive/HSE Final Project/Data/sales_train.csv.zip'
! unzip '/content/drive/MyDrive/HSE Final Project/Data/test.csv.zip'

Archive:  /content/drive/MyDrive/HSE Final Project/Data/items.csv.zip
  inflating: items.csv               
Archive:  /content/drive/MyDrive/HSE Final Project/Data/sales_train.csv.zip
  inflating: sales_train.csv         
Archive:  /content/drive/MyDrive/HSE Final Project/Data/test.csv.zip
  inflating: test.csv                


In [ ]:
! mkdir '/content/drive/MyDrive/HSE Final Project/Data/Zip'

In [ ]:
filenames = ['items.csv.zip', 'sales_train.csv.zip', 'test.csv.zip']

src = '/content/drive/MyDrive/HSE Final Project/Data/'
dest = src + 'Zip/'

for file_name in filenames:
    shutil.move(src + file_name, dest + file_name)

In [ ]:
file_names = [file_name[:-4] for file_name in filenames]
print(file_names)

['items.csv', 'sales_train.csv', 'test.csv']


In [ ]:
src = '/content/'
dest = '/content/drive/MyDrive/HSE Final Project/Data/'

for file_name in file_names:
    shutil.move(src + file_name, dest + file_name)

# Review criteria

**Clarity**

- The clear step-by-step instruction on how to produce the final submit file is provided

- Code has comments where it is needed and meaningful function names

**Feature preprocessing and generation with respect to models**

- Several simple features are generated

- For non-tree-based models preprocessing is used or the absence of it is explained

**Feature extraction from text and images**

- Features from text are extracted

- Special preprocessings for text are utilized (TF-IDF, stemming, levenshtening...)

**EDA**

- Several interesting observations about data are discovered and explained

- Target distribution is visualized, time trend is assessed

**Validation**

- Type of train/test split is identified and used for validation

- Type of public/private split is identified

**Data leakages**

- Data is investigated for data leakages and investigation process is described

- Found data leakages are utilized

**Metrics optimization**

- Correct metric is optimized

**Advanced Features I: mean encodings**

- Mean-encoding is applied

- Mean-encoding is set up correctly, i.e. KFold or expanding scheme are utilized correctly

**Advanced Features II**

- At least one feature from this topic is introduced

**Hyperparameter tuning**

- Parameters of models are roughly optimal

**Ensembles**

- Ensembling is utilized (linear combination counts)

- Validation with ensembling scheme is set up correctly, i.e. KFold or Holdout is utilized

- Models from different classes are utilized (at least two from the following: KNN, linear models, RF, GBDT, NN)

# Baseline

## Main Data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
dest = '/content/drive/MyDrive/HSE Final Project/Data/'

df_train = pd.read_csv(dest + 'sales_train.csv')
df_test = pd.read_csv(dest + 'test.csv')

In [4]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [5]:
df_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


## Additional Data (not needed yet)

In [ ]:
df_shops = pd.read_csv(dest + 'shops.csv')
df_shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [ ]:
df_categories = pd.read_csv(dest + 'item_categories.csv')
df_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


## Analysis for train

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [7]:
df_train['date'] =  pd.to_datetime(df_train['date'], format='%d.%m.%Y')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   date            datetime64[ns]
 1   date_block_num  int64         
 2   shop_id         int64         
 3   item_id         int64         
 4   item_price      float64       
 5   item_cnt_day    float64       
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 134.4 MB


In [8]:
df_train['year'] = df_train.date.dt.year
df_train['month'] = df_train.date.dt.month
df_train['day'] = df_train.date.dt.day
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,month,day
0,2013-01-02,0,59,22154,999.00,1.0,2013,1,2
1,2013-01-03,0,25,2552,899.00,1.0,2013,1,3
2,2013-01-05,0,25,2552,899.00,-1.0,2013,1,5
3,2013-01-06,0,25,2554,1709.05,1.0,2013,1,6
4,2013-01-15,0,25,2555,1099.00,1.0,2013,1,15


In [9]:
df_train.drop('date', axis=1, inplace=True)
df_train.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,month,day
0,0,59,22154,999.00,1.0,2013,1,2
1,0,25,2552,899.00,1.0,2013,1,3
2,0,25,2552,899.00,-1.0,2013,1,5
3,0,25,2554,1709.05,1.0,2013,1,6
4,0,25,2555,1099.00,1.0,2013,1,15


In [10]:
df_train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,month,day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00,2.013777e+03,6.247717e+00,1.585267e+01
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00,7.684790e-01,3.536219e+00,8.923483e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01,2.013000e+03,1.000000e+00,1.000000e+00
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00,2.013000e+03,3.000000e+00,8.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00,2.014000e+03,6.000000e+00,1.600000e+01
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00,2.014000e+03,9.000000e+00,2.400000e+01
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03,2.015000e+03,1.200000e+01,3.100000e+01


In [15]:
df_train_monthly = df_train.groupby(['date_block_num', 'item_id', 'shop_id'])['item_cnt_day'].sum()
df_train_monthly.head()

date_block_num  item_id  shop_id
0               19       25         1.0
                27       1          1.0
                         2          1.0
                         10         1.0
                         19         1.0
Name: item_cnt_day, dtype: float64

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split